### Importing librairies

In [1]:
import pandas as pd

# External librairies
# Google trend unofficial API
from pytrends.request import TrendReq

data_folder = './Data/'

In [ ]:
def extract(dict_in, list_out, list_out2):
    '''
    Recursive function extracting all integers and entry with key 'name' contained in a nested dictionary with list.
    code adapted from here : https://stackoverflow.com/questions/10569636/how-to-get-all-keysvalues-in-nested-dict-of-list-of-dicts-and-dicts/10569687
    
    INPUT:
        - dict_in: dictionary you want to extract from
    
    OUTPUT:
        - list_out: list containing all integers entry in dict_in
        - list_out2: list containing all entry entry with key 'name' in dict_in
    '''
    for key, value in dict_in.items():               # for all entry in dict_in
        if isinstance(value, dict):                       # If value itself is dictionary
            extract(value, list_out, list_out2)                # Then do recursion and input this dictionary in dict_in
        elif isinstance(value, list):                     # Else if this value is a list
            for key2 in value:                                 # Then for each entry in this list
                extract(key2, list_out, list_out2)                  # Fo recursion and input this entry in dict_in
        elif isinstance(value, int):                      # Else if this value is an integer
            list_out.append(value)                             # add the value to list_out
        elif key=='name':                                 # Else if this value has key == 'name'
            list_out2.append(value)                            # add this value to list_out2
    return list_out,list_out2                        # Return the two list to have valid recursion

In [2]:
# Create a trend rquest object with language, timzone offset, number of retry if request fail, time factor to make each retry (wait 0.1s , 0.2s, 0.3s, ...)
pytrends = TrendReq(hl='fr-FR',tz=60, retries=10,backoff_factor=0.1,)

# Extract all google trend categories in a nested dictionary
categories_dictionary = pytrends.categories()

# initailaze list for extracting data
categories_ids =[]
categories_names = []
# Extracting all the data in nested dictionary with function
[categories_ids,categories_names] = extract(categories_dictionary,categories_ids,categories_names)

# making a dataframe and drooping duplicates categories (ex: category Programmtion id: 31 is a sub category of 'Computer Hardware' and 'COmputer science')
categories_df = pd.DataFrame(zip(categories_ids,categories_names),columns=['id','name']).sort_values('id')
categories_df=categories_df.drop_duplicates(subset ="id")

# saving to a csv a getting an overview
categories_df.to_csv(data_folder + 'categories.csv')
categories_df

,id,name
1426,0,Toutes catégories
240,3,Arts et divertissements
715,5,Informatique et électronique
396,7,Finance
785,8,Jeux
...,...,...
612,1393,Décodeurs et magnétoscopes numériques
614,1394,Lecteurs et enregistreurs Blu-Ray
615,1395,Lecteurs et enregistreurs DVD
603,1396,Oreillettes


In [ ]:
# initializing a data frame
GTD = pd.DataFrame()
# for each categories
for j,i in enumerate(categories_df['id']):
    # build request empty key word with france geolocation with the jth category from 2004-01-01 to 2020-12-31
    kw_list = [""]
    pytrends.build_payload(kw_list, geo='FR', cat=i , timeframe='2004-01-01 2020-12-31')
    
    # getting google trend data
    temp = pytrends.interest_over_time()
    
    # if the return is not empty save data (may happen for small categories ex : category 42 "jazz" https://trends.google.com/trends/explore?cat=42&date=all&geo=FR)
    if not temp.empty:
        GTD[i]=temp.iloc[:,0]   
    
# getting an overview    
GTD

In [15]:
#export data in csv
GTD.to_csv(data_folder+ dt_string + 'GTD.csv')